# Exámen Diplomatura Ciencia de Datos 4º Bimestre

Este notebook contiene el analisis exploratorio y la limpieza de los datos a aplicar en el modelo predictivo para el problema de negocio planteado.

## Datos del Grupo

Nombre y Apellido | LU

---

In [129]:
import pandas as pd
import numpy as np

In [130]:
df = pd.read_excel("./db/budget_raw.xlsx")
# df = pd.read_csv("./db/budget_raw.csv",encoding='latin')
df.sample(5)

,Carryover 2021,mes num,tipo imputacion,tipo de gasto,pais,area,budget disponible,year,Es Extrabudget,budget,budget int,saldo,saldo int,real,real int
2215,NaN,10,Opex,GASTO21,Italia,CECO2,Con Budget,2021,No,3556.0,3556,0.0,0,0.00,0
1259,NaN,5,Opex,GASTO21,España,CECO2,Con Budget,2020,No,6632.0,6632,0.0,0,0.00,0
9141,NaN,10,Capex,GASTO12,Francia,CECO1,Sin Budget,2021,NaN,0.0,0,0.0,0,6598.44,6598
6686,NaN,9,Opex,GASTO13,United Kingdom,CECO1,Con Budget,2022,NO,7400.0,7400,0.0,0,0.00,0
8818,NaN,4,Opex,GASTO15,Francia,CECO2,Con Budget,2022,NaN,1017.0,1017,0.0,0,0.00,0


Quitamos un par de columnas vestigio que quedaron del ETL

In [131]:
df = df.drop(['budget','real','saldo','Carryover 2021'],axis=1)

Renombramos las coluimnas a algo mas útil 

In [132]:
df['budget'] = df['budget int']
df['real'] = df['real int']
df['saldo'] = df['saldo int']
df['es_extrabudget'] = df['Es Extrabudget']
df = df.drop(['budget int','real int','saldo int','Es Extrabudget'],axis=1)
df.sample()

,mes num,tipo imputacion,tipo de gasto,pais,area,budget disponible,year,budget,real,saldo,es_extrabudget
7307,8,Opex,GASTO2,Polonia,CECO2,Con Budget,2021,545,0,0,No


In [133]:
df["gasto"] = df['tipo de gasto']
df['budget_disponible'] = df['budget disponible']
df['imputacion'] = df['tipo imputacion']
df['mes'] = df['mes num']
df = df.drop(['tipo de gasto','budget disponible','tipo imputacion','mes num'],axis=1)

In [134]:
df.sample(5)

,pais,area,year,budget,real,saldo,es_extrabudget,gasto,budget_disponible,imputacion,mes
4444,Francia,CECO1,2022,2000,0,0,NaN,GASTO16,Con Budget,Opex,12
9765,United Kingdom,CECO2,2020,15300,0,0,No,GASTO3,Con Budget,Opex,2
639,Polonia,CECO2,2022,0,23829,0,NaN,GASTO17,Sin Budget,Capex,3
7001,Alemania,CECO2,2021,91500,0,0,No,GASTO13,Con Budget,Opex,5
2414,Italia,CECO1,2022,0,3052,0,NaN,GASTO10,Sin Budget,Capex,3


In [135]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10233 entries, 0 to 10232
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   pais               10233 non-null  object
 1   area               10233 non-null  object
 2   year               10233 non-null  int64 
 3   budget             10233 non-null  int64 
 4   real               10233 non-null  int64 
 5   saldo              10233 non-null  int64 
 6   es_extrabudget     4770 non-null   object
 7   gasto              10233 non-null  object
 8   budget_disponible  10233 non-null  object
 9   imputacion         10233 non-null  object
 10  mes                10233 non-null  int64 
dtypes: int64(5), object(6)
memory usage: 879.5+ KB


In [136]:
df.isna().sum()

pais                    0
area                    0
year                    0
budget                  0
real                    0
saldo                   0
es_extrabudget       5463
gasto                   0
budget_disponible       0
imputacion              0
mes                     0
dtype: int64

In [137]:
# valores unicos en todas las columnas
for i in df.columns:
    print(f'{i}:{df[i].unique()}\n')

pais:['España' 'Francia' 'United Kingdom' 'Italia' 'Republica Checa' 'Alemania'
 'Polonia' 'Suiza' 'Austria' 'Eslovenia']

area:['CECO1' 'CECO2' 'CECO3']

year:[2022 2021 2020]

budget:[      0    3550   13337 ... 2017774  119903    9400]

real:[522187   1028      0 ... 115495   2923   3138]

saldo:[      0   31949   19236   10614   58727  471039   16493    3953  293136
     230    2470   69279     688     944   33348    7136   13052     555
      30      23     565     537    1114    9955    2637   13159  150758
   41560  585558  313447    1153    2237    7692     120 2573725    7727
   47729    2203    5963    5485    6025  155709  382340     226   10146
    6562   67686     800  614335    8383 1700076  128134    2230    7213
  509409     380    5228   14490    4015     545   12849   19412     913
    2208   14103    7830  165749    1779  513608     390   49338  531585
   11847  434662   99380     143    2775     462    4840    4803    2968
    4075  203048   44310     634  666901   

------

## Transformar a "Booleanos"

Remplazamos los campos donde dice Sin Budget por 0s y el resto por 1s

In [138]:
dfBool = df.copy()
mask = df['budget_disponible'] == "Sin Budget"
dfBool.loc[mask,"budget_disponible"] = 0
mask = df['budget_disponible'] == "Con Budget"
dfBool.loc[mask,"budget_disponible"] = 1
dfBool['budget_disponible'] = dfBool['budget_disponible'].astype(int)
dfBool.sample(5)


,pais,area,year,budget,real,saldo,es_extrabudget,gasto,budget_disponible,imputacion,mes
5043,United Kingdom,CECO2,2022,833,0,0,NO,GASTO12,1,Opex,1
8911,United Kingdom,CECO2,2020,10982,0,0,No,GASTO9,1,Opex,3
7557,Italia,CECO1,2020,0,-301,0,NaN,GASTO15,0,Opex,6
1069,España,CECO2,2020,0,0,0,No,GASTO6,1,Opex,1
1859,Republica Checa,CECO2,2020,166,0,0,No,GASTO20,1,Opex,1


In [139]:
# dfBool['Carryover 2021'] = np.where(dfBool['Carryover 2021'] == 'Carryover 2021',1,0) # CORRER SOLO 1 VEZ


In [140]:
# print(df['Carryover 2021'].value_counts(), "\n dfbool:  ",dfBool['Carryover 2021'].value_counts())



In [141]:
# dfBool[dfBool['Carryover 2021'] == 1]

Cambiamos los valores de la col es_extrabudget a boleanos

In [ ]:
# [nan 'NO' 'SI' 51406 'AJUSTADO' 'ESTA OK' 'SÍ']
# [nan 'No' 'SÍ' 'SI' 'NO' 'Si' 'ESTA OK' '51406' 'AJUSTADO']
dfBool['es_extrabudget'] = dfBool['es_extrabudget'].replace(np.nan, 0)
dfBool.loc[dfBool['es_extrabudget'] == 'NO',"es_extrabudget"] = 0
dfBool.loc[dfBool['es_extrabudget'] == 'No',"es_extrabudget"] = 0
dfBool.loc[dfBool['es_extrabudget'] == 'SI',"es_extrabudget"] = 1
dfBool.loc[dfBool['es_extrabudget'] == 'SÍ',"es_extrabudget"] = 1 
dfBool.loc[dfBool['es_extrabudget'] == 'Si',"es_extrabudget"] = 1 
dfBool.loc[dfBool['es_extrabudget'] == 'AJUSTADO',"es_extrabudget"] = 1   
dfBool.loc[dfBool['es_extrabudget'] == 'ESTA OK',"es_extrabudget"] = 1   
dfBool.loc[dfBool['es_extrabudget'] == 51406,"es_extrabudget"] = 1   
dfBool.loc[dfBool['es_extrabudget'] == '51406',"es_extrabudget"] = 1   



In [143]:
dfBool['es_extrabudget'].value_counts()

0    10020
1      213
Name: es_extrabudget, dtype: int64

In [144]:
"""
    Convertimos el tipo de datos de la columnas a Ints
"""
dfBool['es_extrabudget']=dfBool['es_extrabudget'].astype(int)
dfBool.dtypes

pais                 object
area                 object
year                  int64
budget                int64
real                  int64
saldo                 int64
es_extrabudget        int32
gasto                object
budget_disponible     int32
imputacion           object
mes                   int64
dtype: object

In [145]:
"""
    Copiamos el dataframe 
"""
dfOut = dfBool.copy()

## Area y Cuenta

### Area

Convertimos la columna area a numeros

In [146]:
dfOut.loc[dfOut.area == 'CECO1','area'] = 1
dfOut.loc[dfOut.area == 'CECO2','area'] = 2
dfOut.loc[dfOut.area == 'CECO3','area'] = 3

In [147]:
dfOut.area.unique()

array([1, 2, 3], dtype=object)

In [148]:
dfOut['area']=dfOut['area'].astype(int)
dfOut.dtypes

pais                 object
area                  int32
year                  int64
budget                int64
real                  int64
saldo                 int64
es_extrabudget        int32
gasto                object
budget_disponible     int32
imputacion           object
mes                   int64
dtype: object

### Cuenta

Truncamos los caracteres de la columna gasto y transformamos el tipo de datos a enteros

In [149]:
dfOut.sample(5)

,pais,area,year,budget,real,saldo,es_extrabudget,gasto,budget_disponible,imputacion,mes
8927,Polonia,2,2022,0,609,27329,0,GASTO3,0,Opex,11
6608,Francia,2,2020,0,244,0,0,GASTO16,0,Opex,1
7398,Francia,2,2022,0,1977,0,0,GASTO2,0,Opex,9
3891,Polonia,2,2020,101850,0,0,0,GASTO11,1,Capex,9
1560,Francia,2,2021,0,4534,0,0,GASTO19,0,Opex,11


In [150]:
dfOut['gasto'] = dfOut.gasto.str.replace('GASTO','')
dfOut['gasto'] = dfOut.gasto.str.replace('SIN CTA','0')


In [151]:
dfOut.gasto.unique()

array(['13', '6', '5', '9', '15', '12', '8', '18', '14', '7', '17', '19',
       '2', '21', '11', '20', '4', '16', '3', '0', '1', '10'],
      dtype=object)

In [152]:
dfOut.gasto=dfOut.gasto.astype(int)
dfOut.dtypes

pais                 object
area                  int32
year                  int64
budget                int64
real                  int64
saldo                 int64
es_extrabudget        int32
gasto                 int32
budget_disponible     int32
imputacion           object
mes                   int64
dtype: object

In [153]:
dfOut.sample(10)

,pais,area,year,budget,real,saldo,es_extrabudget,gasto,budget_disponible,imputacion,mes
5563,United Kingdom,3,2021,1750,0,0,0,2,1,Opex,8
2002,Polonia,2,2022,0,10552,0,0,19,0,Opex,10
8676,Francia,2,2021,0,0,0,0,7,1,Opex,8
4605,Alemania,2,2021,5000,0,0,0,4,1,Opex,8
8119,Alemania,1,2021,0,32200,0,0,20,0,Opex,9
9382,Alemania,1,2022,900,0,0,0,16,1,Opex,9
2066,Polonia,2,2022,2800,0,0,0,12,1,Capex,5
9062,España,2,2021,19311,0,0,0,9,1,Opex,8
5147,Alemania,1,2021,0,29371,0,0,13,0,Capex,10
9835,Italia,1,2021,0,4260,0,0,21,0,Opex,7


## Pais

In [154]:
# dfOut.loc[dfOut.pais=='USA','pais'] = 'Estados Unidos'

In [155]:
dfOut.pais.unique()

array(['España', 'Francia', 'United Kingdom', 'Italia', 'Republica Checa',
       'Alemania', 'Polonia', 'Suiza', 'Austria', 'Eslovenia'],
      dtype=object)

## Budget, real y Saldo

Convertimos las columnas Budget, real y saldo a columnas numericas y llenamos los NAN con ceros

In [156]:
# errors Coerce Transforma los errores a nan
dfOut['budget'] = pd.to_numeric(dfOut['budget'])
dfOut['real'] = pd.to_numeric(dfOut['real'])
dfOut['saldo'] = pd.to_numeric(dfOut['saldo'])
dfOut.dtypes

pais                 object
area                  int32
year                  int64
budget                int64
real                  int64
saldo                 int64
es_extrabudget        int32
gasto                 int32
budget_disponible     int32
imputacion           object
mes                   int64
dtype: object

In [157]:
dfOut['budget'] = dfOut['budget'].fillna(0)
dfOut['real']   = dfOut['real'].fillna(0)
dfOut['saldo']  =dfOut['saldo'].fillna(0)

In [158]:
dfOut.isna().sum()

pais                 0
area                 0
year                 0
budget               0
real                 0
saldo                0
es_extrabudget       0
gasto                0
budget_disponible    0
imputacion           0
mes                  0
dtype: int64

Ordenamos por año y mes

In [159]:
dfOut = dfOut.set_index(['year','mes'])
dfOut = dfOut.sort_values(by=["year",'mes'])
dfOut = dfOut.reset_index()

In [162]:
dfOut

,year,mes,pais,area,budget,real,saldo,es_extrabudget,gasto,budget_disponible,imputacion
0,2020,1,Suiza,2,1125,0,0,0,16,1,Opex
1,2020,1,Francia,2,15000,0,0,0,11,1,Capex
2,2020,1,Polonia,2,0,1761,0,0,8,0,Capex
3,2020,1,United Kingdom,2,0,0,0,0,8,1,Capex
4,2020,1,Austria,3,107500,0,0,0,19,1,Opex
...,...,...,...,...,...,...,...,...,...,...,...
10228,2022,12,Republica Checa,2,200,0,0,0,16,1,Opex
10229,2022,12,Alemania,2,20672,0,0,0,13,1,Opex
10230,2022,12,United Kingdom,1,22000,0,0,1,13,1,Capex
10231,2022,12,España,1,1838610,0,0,0,13,1,Capex


In [160]:
dfOut.to_csv('db/budget_limpio.csv')

-----

In [161]:
# Tipos de archivos unicos en todas las columans

def CollumnTypes(var):
    arr = []
    for i in var:
        arr.append(type(i))
    
    return arr

for i in dfOut.columns:
    print(f"{i}: {set(CollumnTypes(dfOut[i]))}")


year: {<class 'int'>}
mes: {<class 'int'>}
pais: {<class 'str'>}
area: {<class 'int'>}
budget: {<class 'int'>}
real: {<class 'int'>}
saldo: {<class 'int'>}
es_extrabudget: {<class 'int'>}
gasto: {<class 'int'>}
budget_disponible: {<class 'int'>}
imputacion: {<class 'str'>}
